In [1]:
import nltk 
nltk.download("gutenberg")
from nltk.corpus import gutenberg
import pandas as pd

##LOAD THE DATASET

data = gutenberg.raw("shakespeare-hamlet.txt")
with open("hemlet.txt","w") as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

with open("hemlet.txt","r") as file:
    text = file.read().lower()
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words    

4818

In [3]:
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
input_sequences

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [5]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len 

14

In [6]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen =max_sequence_len,padding="pre"))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [7]:
import tensorflow as tf
x,y = input_sequences[:,:-1],input_sequences[:,-1]

In [8]:
y = tf.keras.utils.to_categorical(y,num_classes = total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [10]:
## Train our LSTM Rnn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

model = Sequential()
model.add(Embedding(total_words,100,input_length = max_sequence_len-1))
model.add(LSTM(150,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation = "softmax"))

#Compile the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 lstm (LSTM)                 (None, 13, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
history = model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1)

Epoch 1/100


644/644 [==============================] - 38s 45ms/step - loss: 6.9282 - accuracy: 0.0343 - val_loss: 6.7594 - val_accuracy: 0.0311
Epoch 2/100
644/644 [==============================] - 30s 46ms/step - loss: 6.4792 - accuracy: 0.0379 - val_loss: 6.8654 - val_accuracy: 0.0396
Epoch 3/100
644/644 [==============================] - 30s 46ms/step - loss: 6.3345 - accuracy: 0.0455 - val_loss: 6.8649 - val_accuracy: 0.0457
Epoch 4/100
644/644 [==============================] - 30s 46ms/step - loss: 6.1843 - accuracy: 0.0523 - val_loss: 6.8956 - val_accuracy: 0.0439
Epoch 5/100
644/644 [==============================] - 29s 46ms/step - loss: 6.0434 - accuracy: 0.0556 - val_loss: 6.9121 - val_accuracy: 0.0556
Epoch 6/100
644/644 [==============================] - 30s 46ms/step - loss: 5.8996 - accuracy: 0.0654 - val_loss: 6.9675 - val_accuracy: 0.0556
Epoch 7/100
644/644 [==============================] - 30s 47ms/step - loss: 5.7556 - accuracy: 0.0739 - val_loss: 7.0256 - val_

In [28]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list],maxlen = max_sequence_len-1,padding="pre")
    predicted = model.predict(token_list, verbose = 0) 
    predict_next_index = np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index == predict_next_index:
            return word    
    return None

In [42]:
input_text = "Their was a boy who"
max_sequence_len = model.input_shape[1]+1
# next_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
next_word = predict_next_word(model,tokenizer,input_text, max_sequence_len)
print(f"Next Word Prediction: {next_word}")


Next Word Prediction: brought


In [22]:
model.save("Next_word_Rnn.h5")

z:\code\LC\03\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
import pickle
with open("tokenizer.pickle","wb") as handle:
    pickle.dump(tokenizer,handle, protocol=pickle.HIGHEST_PROTOCOL)